# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,6.37,91,100,17.12,TF,1715303885
1,1,fort bragg,35.1390,-79.0060,23.08,82,0,4.12,US,1715303885
2,2,adamstown,-25.0660,-130.1015,22.29,76,99,4.79,PN,1715303794
3,3,mont-tremblant,46.1949,-74.6264,7.98,87,73,0.51,CA,1715303837
4,4,invercargill,-46.4000,168.3500,9.45,77,100,5.36,NZ,1715303887


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    scale = 0.6,
   )

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) &
                                  (city_data_df["Max Temp"] < 27) &
                                  (city_data_df["Wind Speed"] >4.5) &
                                  (city_data_df["Humidity"] <=75)]

# Drop any rows with null values
narrow_city_df.dropna()

# Display sample data
narrow_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
74,74,geraldton,-28.7667,114.6000,24.23,46,100,5.66,AU,1715303735,
98,98,port mathurin,-19.6833,63.4167,25.38,72,100,11.43,MU,1715303939,
100,100,hurricane,37.1753,-113.2899,24.19,37,29,8.56,US,1715303940,
128,128,runing,33.0011,114.3542,24.44,46,4,7.40,CN,1715303955,
150,150,siwa oasis,29.2041,25.5195,25.01,33,99,7.30,EG,1715303967,
169,169,cidade velha,14.9167,-23.6167,24.26,60,0,5.66,CV,1715303977,
170,170,isangel,-19.5500,169.2667,26.06,61,75,4.63,VU,1715303977,
269,269,cannonvale,-20.2833,148.7000,23.66,71,100,7.32,AU,1715304035,
272,272,itoman,26.1247,127.6694,25.43,56,75,5.14,JP,1715304037,
339,339,duobao,30.6700,112.6895,22.26,67,100,5.78,CN,1715304077,


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
74,geraldton,AU,-28.7667,114.6000,46,
98,port mathurin,MU,-19.6833,63.4167,72,
100,hurricane,US,37.1753,-113.2899,37,
128,runing,CN,33.0011,114.3542,46,
150,siwa oasis,EG,29.2041,25.5195,33,
169,cidade velha,CV,14.9167,-23.6167,60,
170,isangel,VU,-19.5500,169.2667,61,
269,cannonvale,AU,-20.2833,148.7000,71,
272,itoman,JP,26.1247,127.6694,56,
339,duobao,CN,30.6700,112.6895,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'limit':20,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{lng},{lat},{radius}" 
    params["bias"] =f"proximity:{lng},{lat}" 
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
geraldton - nearest hotel: Ocean Centre Hotel
port mathurin - nearest hotel: Escale Vacances
hurricane - nearest hotel: Wingate by Wyndham Hurricane Near Zion National Park
runing - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
cidade velha - nearest hotel: Encosta Azul
isangel - nearest hotel: Tanna Lodge
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
itoman - nearest hotel: No hotel found
duobao - nearest hotel: No hotel found
salinas - nearest hotel: The Traveler's Hotel
minggang - nearest hotel: No hotel found
souillac - nearest hotel: Hotel SkylineInn
andergrove - nearest hotel: No hotel found
peace river - nearest hotel: New Ridge Inn
bowen - nearest hotel: Whitsunday Sands Resort
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
al jawf - nearest hotel: No hotel found
cooper's town - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
74,geraldton,AU,-28.7667,114.6000,46,Ocean Centre Hotel
98,port mathurin,MU,-19.6833,63.4167,72,Escale Vacances
100,hurricane,US,37.1753,-113.2899,37,Wingate by Wyndham Hurricane Near Zion Nationa...
128,runing,CN,33.0011,114.3542,46,No hotel found
150,siwa oasis,EG,29.2041,25.5195,33,فندق الكيلانى
169,cidade velha,CV,14.9167,-23.6167,60,Encosta Azul
170,isangel,VU,-19.5500,169.2667,61,Tanna Lodge
269,cannonvale,AU,-20.2833,148.7000,71,Whitsundays Rainforest Retreat
272,itoman,JP,26.1247,127.6694,56,No hotel found
339,duobao,CN,30.6700,112.6895,67,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    scale = 0.6,
    hover_cols = ['Country','Hotel Name']
   )

# Display the map
hotel_map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)